In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import datetime
from Models.Transformer import Transformer
from Models.ESBNTransformer import ESBNTransformer2
from Utils.Logging import BatchLoggingModel
import tensorboard

/Users/jonathankonig/miniforge3/envs/esbn_abstractor/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
train_raw, test_raw = tfds.load("scan/addprim_jump", split=["train", "test"])

def standardize(text):
    text = tf.strings.join(["<SOS>", text, "<EOS>"], separator=' ')
    return text


command_processor = tf.keras.layers.TextVectorization(
    standardize=standardize
)
action_processor = tf.keras.layers.TextVectorization(
    standardize=standardize
)

command_processor.adapt(train_raw.map(lambda input: input["commands"]))
action_processor.adapt(train_raw.map(lambda input: input["actions"]))

def process_scan(pair):
    command = command_processor(pair["commands"])
    action = action_processor(pair["actions"])
    action_in = action[:, :-1]
    action_out = action[:, 1:]
    return (command, action_in), action_out

train_raw = train_raw.batch(32, drop_remainder=True).prefetch(20)
test_raw = test_raw.batch(32, drop_remainder=True).prefetch(20)

train_ds = train_raw.map(process_scan, tf.data.AUTOTUNE)
val_ds = test_raw.map(process_scan, tf.data.AUTOTUNE)


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089
2024-03-29 16:43:05.615505: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


In [3]:
for (command, action_in), action_out in train_ds.take(1):
    print(command.shape)
    print(action_in.shape)
    print(action_out.shape)

(32, 11)
(32, 41)
(32, 41)


In [4]:
num_layers = 2
d_model = 32
dff = 64
num_heads = 4
dropout_rate = 0.1

esbn_transformer = ESBNTransformer2(
    num_layers=num_layers,
    d_model=d_model,
    num_heads=num_heads,
    dff=dff,
    input_vocab_size=len(command_processor.get_vocabulary()),
    target_vocab_size=len(action_processor.get_vocabulary()),
    dropout_rate=dropout_rate)

In [5]:
def masked_loss(label, pred):
  mask = label != 0
  loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')
  loss = loss_object(label, pred)

  mask = tf.cast(mask, dtype=loss.dtype)
  loss *= mask

  loss = tf.reduce_sum(loss)/tf.reduce_sum(mask)
  return loss


def masked_accuracy(label, pred):
  pred = tf.argmax(pred, axis=2)
  label = tf.cast(label, pred.dtype)
  match = label == pred

  mask = label != 0

  match = match & mask

  match = tf.cast(match, dtype=tf.float32)
  mask = tf.cast(mask, dtype=tf.float32)
  return tf.reduce_sum(match)/tf.reduce_sum(mask)

In [6]:
esbn_transformer.compile(
    loss=masked_loss,
    optimizer="Adam",
    metrics=[masked_accuracy])

In [7]:
tf.config.run_functions_eagerly(True)
log_dir = "logs/fit/scan_transformer/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
esbn_transformer.fit(train_ds,
                epochs=20,
                validation_data=val_ds,
                callbacks=[tensorboard_callback])

Epoch 1/20
458/458 [==============================] - 143s 313ms/step - loss: 0.8505 - masked_accuracy: 0.6645 - val_loss: 2.9161 - val_masked_accuracy: 0.4400
Epoch 2/20
458/458 [==============================] - 143s 312ms/step - loss: 0.5107 - masked_accuracy: 0.7777 - val_loss: 3.2119 - val_masked_accuracy: 0.4790
Epoch 3/20
458/458 [==============================] - 144s 316ms/step - loss: 0.4257 - masked_accuracy: 0.8101 - val_loss: 3.2353 - val_masked_accuracy: 0.4685
Epoch 4/20
458/458 [==============================] - 143s 311ms/step - loss: 0.3467 - masked_accuracy: 0.8521 - val_loss: 3.1180 - val_masked_accuracy: 0.5637
Epoch 5/20
458/458 [==============================] - 141s 309ms/step - loss: 0.2635 - masked_accuracy: 0.8901 - val_loss: 3.5317 - val_masked_accuracy: 0.5670
Epoch 6/20
162/458 [=========>....................] - ETA: 1:19 - loss: 0.2183 - masked_accuracy: 0.9089

KeyboardInterrupt: 

In [8]:
esbn_transformer.summary()

Model: "esbn_transformer2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 encoder (Encoder)           multiple                  42784     
                                                                 
 decoder (Decoder)           multiple                  76288     
                                                                 
 esbn_encoder (ESBNEncoder)  multiple                  52323     
                                                                 
 dense_14 (Dense)            multiple                  650       
                                                                 
Total params: 172,045
Trainable params: 172,045
Non-trainable params: 0
_________________________________________________________________


In [9]:
tf.config.run_functions_eagerly(True)

# Ordinary Transformer
num_layers = 2
d_model = 32
dff = 64
num_heads = 4
dropout_rate = 0.1

transformer = Transformer(
    num_layers=num_layers,
    d_model=d_model,
    num_heads=num_heads,
    dff=dff,
    input_vocab_size=len(command_processor.get_vocabulary()),
    target_vocab_size=len(action_processor.get_vocabulary()),
    dropout_rate=dropout_rate)

transformer.compile(
    loss=masked_loss,
    optimizer="Adam",
    metrics=[masked_accuracy])

log_dir = "logs/fit/scan_transformer/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
transformer.fit(train_ds,
                epochs=20,
                validation_data=val_ds,
                callbacks=[tensorboard_callback])

Epoch 1/20
458/458 [==============================] - 71s 154ms/step - loss: 0.8040 - masked_accuracy: 0.6832 - val_loss: 2.8252 - val_masked_accuracy: 0.4118
Epoch 2/20
458/458 [==============================] - 75s 163ms/step - loss: 0.4918 - masked_accuracy: 0.7875 - val_loss: 3.1255 - val_masked_accuracy: 0.4272
Epoch 3/20
458/458 [==============================] - 73s 160ms/step - loss: 0.4034 - masked_accuracy: 0.8237 - val_loss: 3.2123 - val_masked_accuracy: 0.4817
Epoch 4/20
458/458 [==============================] - 77s 169ms/step - loss: 0.3158 - masked_accuracy: 0.8664 - val_loss: 3.4009 - val_masked_accuracy: 0.5242
Epoch 5/20
458/458 [==============================] - 74s 161ms/step - loss: 0.2287 - masked_accuracy: 0.9048 - val_loss: 3.3856 - val_masked_accuracy: 0.5609
Epoch 6/20
458/458 [==============================] - 74s 162ms/step - loss: 0.1823 - masked_accuracy: 0.9242 - val_loss: 3.3009 - val_masked_accuracy: 0.5676
Epoch 7/20
458/458 [==========================

In [ ]:
transformer.summary()

Model: "transformer"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 encoder_1 (Encoder)         multiple                  85024     
                                                                 
 decoder_1 (Decoder)         multiple                  152256    
                                                                 
 dense_37 (Dense)            multiple                  330       
                                                                 
Total params: 237,610
Trainable params: 237,610
Non-trainable params: 0
_________________________________________________________________
